# Support Vector Machine Regression Model



In [8]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from tempfile import mkdtemp
import warnings
from sklearn.svm import SVR

In [9]:
numerical_df = pd.read_csv("numerical_df.csv")
pd.set_option('display.max_columns', None) 

In [10]:
numerical_df = numerical_df.select_dtypes(exclude=['object'])
numerical_df

,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average,PER Improvement
0,23.0,79.0,72.0,28.4,0.538,0.1,0.000,7.4,0.543,0.538,3.6,0.780,2.7,7.8,1.1,0.9,1.0,1.3,2.2,10.8,220.0,1.0,15.7,0.599,0.009,4.6,1.5,2.0,12.5,14.7,4.3,7.6,0.163,1.0,0.9,1.7,15.3,205.74,0.0,1.0,0.0,13.986667,1.2,0.300000,0.5,0.006667,0.206667,0.539867,9.453333,0.0,14.540000,0.054467,0.481200,-1.2
1,24.0,82.0,64.0,32.1,0.503,0.0,0.000,7.8,0.505,0.503,4.6,0.773,3.0,8.7,1.1,1.1,0.5,1.5,2.5,11.4,220.0,2.0,14.5,0.581,0.003,4.5,1.6,1.0,12.9,14.7,4.5,7.9,0.144,0.3,0.4,1.5,15.3,205.74,0.0,1.0,0.0,13.950000,-3.3,0.235714,-1.4,-0.021429,0.185714,0.536929,9.314286,-2.3,14.528571,0.058143,0.479643,3.3
2,25.0,82.0,82.0,30.6,0.529,0.2,0.235,9.0,0.536,0.532,4.4,0.786,3.1,9.0,1.3,1.1,0.7,1.5,2.1,13.3,220.0,3.0,17.8,0.594,0.022,5.5,1.8,1.2,11.5,17.0,5.8,9.4,0.179,1.6,1.8,2.3,16.4,205.74,0.0,1.0,0.0,13.653846,3.1,0.507692,1.5,-0.161538,0.307692,0.532538,9.007692,-0.1,14.338462,0.060923,0.475846,-3.1
3,26.0,82.0,82.0,33.0,0.478,0.6,0.283,9.3,0.489,0.486,4.5,0.751,3.2,8.7,1.1,0.8,0.6,1.4,2.5,12.9,220.0,4.0,14.7,0.548,0.057,4.6,1.2,1.1,10.7,17.1,4.4,7.7,0.137,-0.6,0.3,1.0,15.1,205.74,0.0,1.0,0.0,13.566667,0.9,0.291667,0.2,-0.200000,0.208333,0.531250,8.683333,1.5,14.108333,0.061250,0.475667,-0.9
4,27.0,82.0,21.0,26.4,0.476,0.7,0.200,5.9,0.507,0.486,3.7,0.738,2.5,6.3,0.9,0.7,0.3,1.2,1.4,9.1,220.0,5.0,13.8,0.556,0.101,4.5,1.4,0.6,12.8,15.6,3.2,6.1,0.135,0.0,0.1,1.1,13.8,205.74,0.0,1.0,0.0,13.545455,-2.9,0.236364,-1.6,-0.227273,0.209091,0.529000,8.645455,-1.2,13.972727,0.057636,0.475636,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11370,23.0,37.0,0.0,12.5,0.482,1.0,0.361,3.6,0.515,0.521,1.7,0.813,0.9,2.7,0.7,0.3,0.1,0.8,1.6,6.2,235.0,1.0,16.5,0.578,0.212,9.2,1.1,0.8,13.1,21.7,0.8,1.2,0.124,-0.5,0.1,0.2,11.6,210.82,0.0,1.0,0.0,16.500000,0.0,0.000000,0.0,0.100000,0.000000,0.578000,6.200000,0.0,21.700000,0.212000,0.482000,-6.6
11371,29.0,74.0,4.0,15.9,0.505,0.0,0.000,5.1,0.505,0.505,2.4,0.771,1.1,3.9,0.5,0.4,1.0,1.1,2.6,6.9,257.0,0.0,16.2,0.569,0.000,5.7,1.3,4.4,15.7,21.4,1.4,3.3,0.134,-2.3,-2.9,-0.1,14.5,213.36,1.0,0.0,0.0,12.350000,5.5,1.100000,0.8,-2.700000,-0.600000,0.608500,5.250000,5.9,15.750000,0.000000,0.555000,-2.2
11372,30.0,30.0,12.0,16.3,0.552,0.0,0.000,4.8,0.552,0.552,1.6,0.792,0.9,3.1,0.3,0.2,0.6,1.0,2.6,6.6,257.0,1.0,14.0,0.596,0.000,3.6,0.7,2.6,14.9,19.2,0.7,1.3,0.133,-2.6,-2.8,-0.1,11.2,213.36,1.0,0.0,0.0,13.633333,-2.2,0.000000,0.1,-2.766667,-0.366667,0.595333,5.800000,-2.2,17.633333,0.000000,0.538333,-4.4
11373,31.0,21.0,2.0,10.6,0.407,0.0,0.000,2.6,0.407,0.407,1.3,0.786,0.9,2.3,0.2,0.2,0.4,0.6,2.1,3.1,257.0,2.0,9.6,0.498,0.000,3.0,1.2,3.1,16.4,16.7,0.1,0.5,0.115,-3.9,-5.1,-0.1,12.8,213.36,1.0,0.0,0.0,13.725000,-4.4,-1.100000,-2.3,-2.775000,-0.850000,0.595500,6.000000,-2.5,18.025000,0.000000,0.541750,4.4


In [11]:
y = numerical_df["PER Improvement"] # Setting our "PER Improvement" as our target variable
X = numerical_df.drop("PER Improvement", axis = 1) #Setting all other features as our data 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 100) # I am splitting the dataset so that we have 90% as trainning set, because we only have 7881 rows so we don't have that many data.

In [12]:
cachedir = mkdtemp()

estimators = [
    ('normalise', StandardScaler()),  # Feature scaling
    ('svm', SVR())  # Support Vector Machine regressor
]

pipe = Pipeline(estimators, memory=cachedir)

param_grid = {
    'svm__C': [1, 2, 3],  # Penalty parameter of the error term
    'svm__kernel': ['linear', 'rbf'],  # Specifies the kernel type to be used in the algorithm
    'svm__gamma': ['scale', 'auto'],  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
}

grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=5, refit=True)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END .svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=  15.8s
[CV 2/5] END .svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=  14.8s
[CV 3/5] END .svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=  14.6s
[CV 4/5] END .svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=  15.2s
[CV 5/5] END .svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=  14.7s
[CV 1/5] END ....svm__C=1, svm__gamma=scale, svm__kernel=rbf; total time=   8.5s
[CV 2/5] END ....svm__C=1, svm__gamma=scale, svm__kernel=rbf; total time=   8.5s
[CV 3/5] END ....svm__C=1, svm__gamma=scale, svm__kernel=rbf; total time=   8.4s
[CV 4/5] END ....svm__C=1, svm__gamma=scale, svm__kernel=rbf; total time=   8.4s
[CV 5/5] END ....svm__C=1, svm__gamma=scale, svm__kernel=rbf; total time=   8.3s
[CV 1/5] END ..svm__C=1, svm__gamma=auto, svm__kernel=linear; total time=  15.5s
[CV 2/5] END ..svm__C=1, svm__gamma=auto, svm__k

GridSearchCV(cv=5,
             estimator=Pipeline(memory='C:\\Users\\ivant\\AppData\\Local\\Temp\\tmpdo4e2wj7',
                                steps=[('normalise', StandardScaler()),
                                       ('svm', SVR())]),
             param_grid={'svm__C': [1, 2, 3], 'svm__gamma': ['scale', 'auto'],
                         'svm__kernel': ['linear', 'rbf']},
             verbose=5)

In [13]:
from sklearn.metrics import accuracy_score
best_params = grid_search.best_params_  # Getting the best parameters from our grid search
print(f"Best parameters: {best_params}") 

SVR_best_pipe = grid_search.best_estimator_

Best parameters: {'svm__C': 2, 'svm__gamma': 'auto', 'svm__kernel': 'rbf'}


The best parameters are tested to be:
C-value of 2,
Auto mode for svm gamma and
rbf for svm kernel

In [14]:
best_validation_performance = grid_search.best_score_  # Getting the best validation set score from our grid search
print(f"Best validation performance: {best_validation_performance}")

y_test_pred = SVR_best_pipe.predict(X_test)  # make predictions on training set 

r2_test = r2_score(y_test, y_test_pred)  # get r2 
print(f'R squared: {round(r2_test, 2)}')

rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))  # get rmse 
print(f'RMSE: {round(rmse_test, 2)}')

mae_test = mean_absolute_error(y_test, y_test_pred)  # get mae
print(f'MAE: {round(mae_test, 2)}')

Best validation performance: 0.325782989116594
R squared: 0.36
RMSE: 2.42
MAE: 1.75


Our SVM model performed not too badly, with a train score of 0.326, Rsquared of 0.36. It has a pretty high RMSE with 2.42, but the lowest MAE with 1.75. This shows that SVM is not too good dealing with the many outliers in my dataset.

Model comparisons will be in the model comparison notebook.